# Import Packages

In [67]:
import json
import os
import pandas as pd
import numpy as np
from datetime import datetime 
import matplotlib.pyplot as plt
import re
from matplotlib.pyplot import figure

# Load Data

In [74]:
output_dir = "output_data/"
file = "response3.pki"
raw_data = pd.read_pickle(output_dir + file)
raw_data = raw_data[["time", "Log ID", "Machine ID", "command", "full_command", "stderr", "stdout"]]
raw_data

,time,Log ID,Machine ID,command,full_command,stderr,stdout
0,2021-04-19 19:34:00,490cd3ad601e875614199fc11aa5ebe2ad67d289cbe519...,10bcd15c,ls,ls,,SuperShell\nstderr.txt
1,2021-04-19 19:35:00,490cd3ad601e875614199fc11aa5ebe2ad67d289cbe519...,10bcd15c,export,export $TEST=Hello,/home/supershell/.offline.sh: line 358: export...,
2,2021-04-19 19:35:00,490cd3ad601e875614199fc11aa5ebe2ad67d289cbe519...,10bcd15c,export,export $TEST='Hello',/home/supershell/.offline.sh: line 358: export...,
3,2021-04-19 19:36:00,490cd3ad601e875614199fc11aa5ebe2ad67d289cbe519...,10bcd15c,export,export PATH=$PATH:$HOME/bin,,
4,2021-04-19 19:36:00,490cd3ad601e875614199fc11aa5ebe2ad67d289cbe519...,10bcd15c,export,export PATH=$PATH:test/bin,,
...,...,...,...,...,...,...,...
468,2021-04-17 19:57:00,00206cd4bdc39bab7e2908eb900370094d721883f30148...,612c4dc4-ce35-4571-952d-cfedc92809d3-5fbf7c6db6-n,ls,ls,,_15881.env\n_15881.variables\nfastx_out\nfx486...
469,2021-04-17 19:57:00,00206cd4bdc39bab7e2908eb900370094d721883f30148...,612c4dc4-ce35-4571-952d-cfedc92809d3-5fbf7c6db6-n,cd,cd ..,,
470,2021-04-17 19:57:00,00206cd4bdc39bab7e2908eb900370094d721883f30148...,612c4dc4-ce35-4571-952d-cfedc92809d3-5fbf7c6db6-n,ls,ls,,job-48632\njob-48633\njob-48634\njob-48635\njo...
471,2021-04-17 19:57:00,00206cd4bdc39bab7e2908eb900370094d721883f30148...,612c4dc4-ce35-4571-952d-cfedc92809d3-5fbf7c6db6-n,pwd,pwd,,/home/jovyan/DNASubway/project-5974


In [75]:
raw_data.describe()

<ipython-input-75-7a57e84b5e52>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  raw_data.describe()


,time,Log ID,Machine ID,command,full_command,stderr,stdout
count,473,473,473,473,455,473,473
unique,166,9,7,31,193,44,140
top,2021-04-19 13:20:00,8067d36662ba1f8b6d7e2281fb338c35f016d2f2e72b2e...,a80ca,ls,ls,,
freq,13,187,187,165,112,423,222
first,2021-04-16 13:38:00,NaN,NaN,NaN,NaN,NaN,NaN
last,2022-09-29 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
print(raw_data["Log ID"].unique())

['490cd3ad601e875614199fc11aa5ebe2ad67d289cbe519e3a41d8fbd2f2af05e'
 '33bb76e6130caab90d89f35542de42c1be3a0f364190f6e3645cd8f03613c1ab'
 '8067d36662ba1f8b6d7e2281fb338c35f016d2f2e72b2e09bb010bd323e89970'
 '2f3c04c9be3cc02268104e419f0c962bc674e4e7411c8c0f67b34f11f89fc398'
 'd255305150a0d892434887745e969de6e48e7d48bc7f84a362951001d7f6c31c'
 '8a0d5bed0152303242b9094ce32098e117fc8f1a7169e69df2cee99ad9d8ec3c'
 '586def0e79609c5554f12ed168ac1962c0d12e7d9ef87a91fdd02406c65dcd24'
 '0f26ab18818b468595373cc657917cfbbedacc14b7d4ff041a49b0aeb5a2f4c6'
 '00206cd4bdc39bab7e2908eb900370094d721883f30148868ba32abb8cdf28b3']


In [77]:
raw_data['Log ID'] = raw_data['Log ID'].replace({'490cd3ad601e875614199fc11aa5ebe2ad67d289cbe519e3a41d8fbd2f2af05e':'User_1', '33bb76e6130caab90d89f35542de42c1be3a0f364190f6e3645cd8f03613c1ab': 'User_2', '8067d36662ba1f8b6d7e2281fb338c35f016d2f2e72b2e09bb010bd323e89970': 'User_3', '2f3c04c9be3cc02268104e419f0c962bc674e4e7411c8c0f67b34f11f89fc398': 'User_4', 'd255305150a0d892434887745e969de6e48e7d48bc7f84a362951001d7f6c31c': 'User_5', '8a0d5bed0152303242b9094ce32098e117fc8f1a7169e69df2cee99ad9d8ec3c': 'User_6', '586def0e79609c5554f12ed168ac1962c0d12e7d9ef87a91fdd02406c65dcd24': 'User_7', '0f26ab18818b468595373cc657917cfbbedacc14b7d4ff041a49b0aeb5a2f4c6': 'User_8', '00206cd4bdc39bab7e2908eb900370094d721883f30148868ba32abb8cdf28b3': 'User_9'})
raw_data.rename(columns={'Log ID': 'User Name'}, inplace=True)
raw_data.head()

,time,User Name,Machine ID,command,full_command,stderr,stdout
0,2021-04-19 19:34:00,User_1,10bcd15c,ls,ls,,SuperShell\nstderr.txt
1,2021-04-19 19:35:00,User_1,10bcd15c,export,export $TEST=Hello,/home/supershell/.offline.sh: line 358: export...,
2,2021-04-19 19:35:00,User_1,10bcd15c,export,export $TEST='Hello',/home/supershell/.offline.sh: line 358: export...,
3,2021-04-19 19:36:00,User_1,10bcd15c,export,export PATH=$PATH:$HOME/bin,,
4,2021-04-19 19:36:00,User_1,10bcd15c,export,export PATH=$PATH:test/bin,,


In [78]:
output_dir = "output_data/"
file = "sd_george.pki"
sam_data = pd.read_pickle(output_dir + file)
sam_data = sam_data[["time", "Log ID", "Machine ID", "command", "full_command", "stderr", "stdout"]]
sam_data.head()

,time,Log ID,Machine ID,command,full_command,stderr,stdout
0,2023-01-19 16:10:00,d04e4ff0c8708760457e1cefad014855b3262f35c85a7a...,a3f8b,ls,ls,,data-store\nstderr.txt\nSuperShell\nwork
1,2023-01-17 19:03:00,cb4db30c2a3bc9fca043f180f62c72c7d1bc8184cd7417...,a07f4,ls,ls /home/jovyan,,data-store\nstderr.txt\nstdout.txt\nSuperShell...
2,2023-01-17 19:03:00,cb4db30c2a3bc9fca043f180f62c72c7d1bc8184cd7417...,a07f4,ls,ls .,,data-store\nstderr.txt\nstdout.txt\nSuperShell...
3,2023-01-17 19:03:00,cb4db30c2a3bc9fca043f180f62c72c7d1bc8184cd7417...,a07f4,/home/jovyan/data-store/data/input/RNA_SEQ_SAM...,/home/jovyan/data-store/data/input/RNA_SEQ_SAM...,/home/jovyan/.offline.sh: line 368: /home/jovy...,
4,2023-01-17 19:03:00,cb4db30c2a3bc9fca043f180f62c72c7d1bc8184cd7417...,a07f4,ls,ls /home/jovyan/data-store/data/input/RNA_SEQ_...,,Drug_A\nDrug_B


In [79]:
print(sam_data["Log ID"].unique())

['d04e4ff0c8708760457e1cefad014855b3262f35c85a7a76c87d1f72539299ec'
 'cb4db30c2a3bc9fca043f180f62c72c7d1bc8184cd74174aa62e5c80059c8c07'
 '6406489831cdc8e2baabf1bb55c770db7402111997feddd153a0e115eec30373'
 'f310fbeac6853eb621fdec11fbf18b37d4c3a6aa9e7f33f41f54ebfa00f38515'
 '7450c9f9ba0d2c95651dffecd26e1fb9d33247117dd9acc897e4ad591582c159']


In [80]:
sam_data['Log ID'] = sam_data['Log ID'].replace({'d04e4ff0c8708760457e1cefad014855b3262f35c85a7a76c87d1f72539299ec': 'Sam', 'cb4db30c2a3bc9fca043f180f62c72c7d1bc8184cd74174aa62e5c80059c8c07': 'John', '6406489831cdc8e2baabf1bb55c770db7402111997feddd153a0e115eec30373': 'Adam', 'f310fbeac6853eb621fdec11fbf18b37d4c3a6aa9e7f33f41f54ebfa00f38515': 'Lily', '7450c9f9ba0d2c95651dffecd26e1fb9d33247117dd9acc897e4ad591582c159': 'Megan'})
sam_data.rename(columns={'Log ID': 'User Name'}, inplace=True)
sam_data

,time,User Name,Machine ID,command,full_command,stderr,stdout
0,2023-01-19 16:10:00,Sam,a3f8b,ls,ls,,data-store\nstderr.txt\nSuperShell\nwork
1,2023-01-17 19:03:00,John,a07f4,ls,ls /home/jovyan,,data-store\nstderr.txt\nstdout.txt\nSuperShell...
2,2023-01-17 19:03:00,John,a07f4,ls,ls .,,data-store\nstderr.txt\nstdout.txt\nSuperShell...
3,2023-01-17 19:03:00,John,a07f4,/home/jovyan/data-store/data/input/RNA_SEQ_SAM...,/home/jovyan/data-store/data/input/RNA_SEQ_SAM...,/home/jovyan/.offline.sh: line 368: /home/jovy...,
4,2023-01-17 19:03:00,John,a07f4,ls,ls /home/jovyan/data-store/data/input/RNA_SEQ_...,,Drug_A\nDrug_B
...,...,...,...,...,...,...,...
341,2023-01-19 14:40:00,Lily,a5cc4,touch,touch ~/data-store/data/input/Discovery/TSVFil...,touch: cannot touch '/home/jovyan/data-store/d...,
342,2022-12-15 19:11:00,Megan,aed0e,clear,clear,,[H[2J[3J
343,2022-12-15 19:11:00,Megan,aed0e,ls,ls,,data\nhome\nstderr.txt\nstdout.txt\nSuperShell
344,2022-12-15 19:14:00,Megan,aed0e,cd,cd,,


In [86]:
frames = [raw_data, sam_data]
combined = pd.concat(frames, ignore_index=True)
combined

,time,User Name,Machine ID,command,full_command,stderr,stdout
0,2021-04-19 19:34:00,User_1,10bcd15c,ls,ls,,SuperShell\nstderr.txt
1,2021-04-19 19:35:00,User_1,10bcd15c,export,export $TEST=Hello,/home/supershell/.offline.sh: line 358: export...,
2,2021-04-19 19:35:00,User_1,10bcd15c,export,export $TEST='Hello',/home/supershell/.offline.sh: line 358: export...,
3,2021-04-19 19:36:00,User_1,10bcd15c,export,export PATH=$PATH:$HOME/bin,,
4,2021-04-19 19:36:00,User_1,10bcd15c,export,export PATH=$PATH:test/bin,,
...,...,...,...,...,...,...,...
814,2023-01-19 14:40:00,Lily,a5cc4,touch,touch ~/data-store/data/input/Discovery/TSVFil...,touch: cannot touch '/home/jovyan/data-store/d...,
815,2022-12-15 19:11:00,Megan,aed0e,clear,clear,,[H[2J[3J
816,2022-12-15 19:11:00,Megan,aed0e,ls,ls,,data\nhome\nstderr.txt\nstdout.txt\nSuperShell
817,2022-12-15 19:14:00,Megan,aed0e,cd,cd,,


**Export for error tree analysis**

In [87]:
output_dir = "error_processed_data/"
to_export = combined.copy()
to_export.to_pickle(output_dir + "error_tree.pki")

# Pandas Processing

## Error Type

In [89]:
unique_error = combined.copy()
unique_error = unique_error[["time", "User Name", "command", "full_command", "stderr"]]

In [90]:
filter = (unique_error["stderr"] != "")

In [91]:
unique_error = unique_error[filter]
unique_error

,time,User Name,command,full_command,stderr
1,2021-04-19 19:35:00,User_1,export,export $TEST=Hello,/home/supershell/.offline.sh: line 358: export...
2,2021-04-19 19:35:00,User_1,export,export $TEST='Hello',/home/supershell/.offline.sh: line 358: export...
18,2022-09-11 18:27:00,User_3,ls,ls /home/jovyan/RNA_SEQ_SAMPLES,ls: cannot access '/home/jovyan/RNA_SEQ_SAMPLE...
22,2022-09-11 18:28:00,User_3,ls,ls /gunings/RNA_SEQ_SAMPLES,ls: cannot access '/gunings/RNA_SEQ_SAMPLES': ...
29,2022-09-11 18:30:00,User_3,cd,cd sdgeorge,/home/jovyan/.offline.sh: line 344: cd: sdgeor...
...,...,...,...,...,...
754,2022-12-01 20:11:00,Adam,ls,ls –l data/input/Discovery,ls: cannot access '–l': No such file or directory
758,2022-12-01 20:17:00,Adam,ls,ls –l ./data/input/RNA_SEQ_SAMPLES stderr.txt,ls: cannot access '–l': No such file or directory
759,2022-12-01 20:17:00,Adam,ls,ls –l ./data/input/RNA_SEQ_SAMPLES stderr.txt,ls: cannot access '–l': No such file or directory
779,2022-12-01 20:23:00,Adam,cd,cd ~/data/input/RNA_SEQ_SAMPLES/Dcd ~/data/inp...,/home/jovyan/.offline.sh: line 354: cd: too ma...


**Expand cell if there are multiple errors in the same command**

In [92]:
def error_splitter(errors):
    return errors.splitlines()

In [93]:
unique_error["error list"] = unique_error["stderr"].apply(lambda x: error_splitter(x))
unique_error


,time,User Name,command,full_command,stderr,error list
1,2021-04-19 19:35:00,User_1,export,export $TEST=Hello,/home/supershell/.offline.sh: line 358: export...,[/home/supershell/.offline.sh: line 358: expor...
2,2021-04-19 19:35:00,User_1,export,export $TEST='Hello',/home/supershell/.offline.sh: line 358: export...,[/home/supershell/.offline.sh: line 358: expor...
18,2022-09-11 18:27:00,User_3,ls,ls /home/jovyan/RNA_SEQ_SAMPLES,ls: cannot access '/home/jovyan/RNA_SEQ_SAMPLE...,[ls: cannot access '/home/jovyan/RNA_SEQ_SAMPL...
22,2022-09-11 18:28:00,User_3,ls,ls /gunings/RNA_SEQ_SAMPLES,ls: cannot access '/gunings/RNA_SEQ_SAMPLES': ...,[ls: cannot access '/gunings/RNA_SEQ_SAMPLES':...
29,2022-09-11 18:30:00,User_3,cd,cd sdgeorge,/home/jovyan/.offline.sh: line 344: cd: sdgeor...,[/home/jovyan/.offline.sh: line 344: cd: sdgeo...
...,...,...,...,...,...,...
754,2022-12-01 20:11:00,Adam,ls,ls –l data/input/Discovery,ls: cannot access '–l': No such file or directory,[ls: cannot access '–l': No such file or direc...
758,2022-12-01 20:17:00,Adam,ls,ls –l ./data/input/RNA_SEQ_SAMPLES stderr.txt,ls: cannot access '–l': No such file or directory,[ls: cannot access '–l': No such file or direc...
759,2022-12-01 20:17:00,Adam,ls,ls –l ./data/input/RNA_SEQ_SAMPLES stderr.txt,ls: cannot access '–l': No such file or directory,[ls: cannot access '–l': No such file or direc...
779,2022-12-01 20:23:00,Adam,cd,cd ~/data/input/RNA_SEQ_SAMPLES/Dcd ~/data/inp...,/home/jovyan/.offline.sh: line 354: cd: too ma...,[/home/jovyan/.offline.sh: line 354: cd: too m...


In [94]:
expand_error = unique_error.explode('error list')
expand_error

,time,User Name,command,full_command,stderr,error list
1,2021-04-19 19:35:00,User_1,export,export $TEST=Hello,/home/supershell/.offline.sh: line 358: export...,/home/supershell/.offline.sh: line 358: export...
2,2021-04-19 19:35:00,User_1,export,export $TEST='Hello',/home/supershell/.offline.sh: line 358: export...,/home/supershell/.offline.sh: line 358: export...
18,2022-09-11 18:27:00,User_3,ls,ls /home/jovyan/RNA_SEQ_SAMPLES,ls: cannot access '/home/jovyan/RNA_SEQ_SAMPLE...,ls: cannot access '/home/jovyan/RNA_SEQ_SAMPLE...
22,2022-09-11 18:28:00,User_3,ls,ls /gunings/RNA_SEQ_SAMPLES,ls: cannot access '/gunings/RNA_SEQ_SAMPLES': ...,ls: cannot access '/gunings/RNA_SEQ_SAMPLES': ...
29,2022-09-11 18:30:00,User_3,cd,cd sdgeorge,/home/jovyan/.offline.sh: line 344: cd: sdgeor...,/home/jovyan/.offline.sh: line 344: cd: sdgeor...
...,...,...,...,...,...,...
754,2022-12-01 20:11:00,Adam,ls,ls –l data/input/Discovery,ls: cannot access '–l': No such file or directory,ls: cannot access '–l': No such file or directory
758,2022-12-01 20:17:00,Adam,ls,ls –l ./data/input/RNA_SEQ_SAMPLES stderr.txt,ls: cannot access '–l': No such file or directory,ls: cannot access '–l': No such file or directory
759,2022-12-01 20:17:00,Adam,ls,ls –l ./data/input/RNA_SEQ_SAMPLES stderr.txt,ls: cannot access '–l': No such file or directory,ls: cannot access '–l': No such file or directory
779,2022-12-01 20:23:00,Adam,cd,cd ~/data/input/RNA_SEQ_SAMPLES/Dcd ~/data/inp...,/home/jovyan/.offline.sh: line 354: cd: too ma...,/home/jovyan/.offline.sh: line 354: cd: too ma...


**First level error type count**

In [95]:
unique_error_1 = len(pd.unique(expand_error["error list"]))
print(unique_error_1)

61


> There are 61 unique errors

**Second level error type count**

In [96]:
second_error = expand_error[["time", "User Name", "command", "full_command", "error list"]]
second_error.rename(columns={'error list': 'error'}, inplace=True)
second_error

/Users/guningshen/Library/Python/3.9/lib/python/site-packages/pandas/core/frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,time,User Name,command,full_command,error
1,2021-04-19 19:35:00,User_1,export,export $TEST=Hello,/home/supershell/.offline.sh: line 358: export...
2,2021-04-19 19:35:00,User_1,export,export $TEST='Hello',/home/supershell/.offline.sh: line 358: export...
18,2022-09-11 18:27:00,User_3,ls,ls /home/jovyan/RNA_SEQ_SAMPLES,ls: cannot access '/home/jovyan/RNA_SEQ_SAMPLE...
22,2022-09-11 18:28:00,User_3,ls,ls /gunings/RNA_SEQ_SAMPLES,ls: cannot access '/gunings/RNA_SEQ_SAMPLES': ...
29,2022-09-11 18:30:00,User_3,cd,cd sdgeorge,/home/jovyan/.offline.sh: line 344: cd: sdgeor...
...,...,...,...,...,...
754,2022-12-01 20:11:00,Adam,ls,ls –l data/input/Discovery,ls: cannot access '–l': No such file or directory
758,2022-12-01 20:17:00,Adam,ls,ls –l ./data/input/RNA_SEQ_SAMPLES stderr.txt,ls: cannot access '–l': No such file or directory
759,2022-12-01 20:17:00,Adam,ls,ls –l ./data/input/RNA_SEQ_SAMPLES stderr.txt,ls: cannot access '–l': No such file or directory
779,2022-12-01 20:23:00,Adam,cd,cd ~/data/input/RNA_SEQ_SAMPLES/Dcd ~/data/inp...,/home/jovyan/.offline.sh: line 354: cd: too ma...


replace file names and directory names with $

In [97]:
file_name_replacer = re.compile(r'.*((\')|(\")).+((\')|(\")).*')
file_name = re.compile(r'((\')|(\")).+((\')|(\"))')

In [98]:
path_replacer = re.compile(r'(\/).+(: line )(\d+).*')
path_name = re.compile(r'(\/).+(: line )(\d+)')

In [99]:
name_between_colon = re.compile(r'.+(:).+(:).+')
colon_name = re.compile(r'(:).+(:)')

In [100]:
cd_replacer = re.compile(r'.*cd:(.+):.*')
cd_name = re.compile(r'cd:(.+):')

In [101]:
def replacer(error):
    if path_replacer.match(error):
        error = re.sub(path_name, 'PATH', error)
    if file_name_replacer.match(error):
        error = re.sub(file_name, '$', error)
    if name_between_colon.match(error):
        error = re.sub(colon_name, ': $:', error)
    return error;
    

In [102]:
second_error["no name"] = second_error["error"].apply(lambda x: replacer(x))
second_error

,time,User Name,command,full_command,error,no name
1,2021-04-19 19:35:00,User_1,export,export $TEST=Hello,/home/supershell/.offline.sh: line 358: export...,PATH: $: not a valid identifier
2,2021-04-19 19:35:00,User_1,export,export $TEST='Hello',/home/supershell/.offline.sh: line 358: export...,PATH: $: not a valid identifier
18,2022-09-11 18:27:00,User_3,ls,ls /home/jovyan/RNA_SEQ_SAMPLES,ls: cannot access '/home/jovyan/RNA_SEQ_SAMPLE...,ls: $: No such file or directory
22,2022-09-11 18:28:00,User_3,ls,ls /gunings/RNA_SEQ_SAMPLES,ls: cannot access '/gunings/RNA_SEQ_SAMPLES': ...,ls: $: No such file or directory
29,2022-09-11 18:30:00,User_3,cd,cd sdgeorge,/home/jovyan/.offline.sh: line 344: cd: sdgeor...,PATH: $: No such file or directory
...,...,...,...,...,...,...
754,2022-12-01 20:11:00,Adam,ls,ls –l data/input/Discovery,ls: cannot access '–l': No such file or directory,ls: $: No such file or directory
758,2022-12-01 20:17:00,Adam,ls,ls –l ./data/input/RNA_SEQ_SAMPLES stderr.txt,ls: cannot access '–l': No such file or directory,ls: $: No such file or directory
759,2022-12-01 20:17:00,Adam,ls,ls –l ./data/input/RNA_SEQ_SAMPLES stderr.txt,ls: cannot access '–l': No such file or directory,ls: $: No such file or directory
779,2022-12-01 20:23:00,Adam,cd,cd ~/data/input/RNA_SEQ_SAMPLES/Dcd ~/data/inp...,/home/jovyan/.offline.sh: line 354: cd: too ma...,PATH: $: too many arguments


In [103]:
unique_error_2 = len(pd.unique(second_error["no name"]))
print(unique_error_2)

24


In [104]:
second_error["no name"].value_counts()

ls: $: No such file or directory              35
cat: $: No such file or directory             10
PATH: $: No such file or directory             9
PATH: $: Is a directory                        4
PATH: $: command not found                     4
cp: target $ is not a directory                2
PATH: $: not a valid identifier                2
cat: $: Too many levels of symbolic links      2
cat: $: Is a directory                         2
diff: $: No such file or directory             2
PATH: $: too many arguments                    2
ln: target $ is not a directory                2
touch: $: Permission denied                    1
Try $ for more information.                    1
cp: -r not specified; omitting directory $     1
mv: $: No such file or directory               1
cp: cannot copy a directory, $                 1
ln: $: File exists                             1
mkdir: $: Permission denied                    1
ln: $: Input/output error                      1
mv: $ are the same f

> Level 2 groupings find that there are 24 unique errors.

**Third level of grouping**

In [105]:
third_error = second_error[["time", "User Name", "command", "full_command", "no name"]]
third_error.rename(columns={'no name': 'error'}, inplace=True)
third_error

/Users/guningshen/Library/Python/3.9/lib/python/site-packages/pandas/core/frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,time,User Name,command,full_command,error
1,2021-04-19 19:35:00,User_1,export,export $TEST=Hello,PATH: $: not a valid identifier
2,2021-04-19 19:35:00,User_1,export,export $TEST='Hello',PATH: $: not a valid identifier
18,2022-09-11 18:27:00,User_3,ls,ls /home/jovyan/RNA_SEQ_SAMPLES,ls: $: No such file or directory
22,2022-09-11 18:28:00,User_3,ls,ls /gunings/RNA_SEQ_SAMPLES,ls: $: No such file or directory
29,2022-09-11 18:30:00,User_3,cd,cd sdgeorge,PATH: $: No such file or directory
...,...,...,...,...,...
754,2022-12-01 20:11:00,Adam,ls,ls –l data/input/Discovery,ls: $: No such file or directory
758,2022-12-01 20:17:00,Adam,ls,ls –l ./data/input/RNA_SEQ_SAMPLES stderr.txt,ls: $: No such file or directory
759,2022-12-01 20:17:00,Adam,ls,ls –l ./data/input/RNA_SEQ_SAMPLES stderr.txt,ls: $: No such file or directory
779,2022-12-01 20:23:00,Adam,cd,cd ~/data/input/RNA_SEQ_SAMPLES/Dcd ~/data/inp...,PATH: $: too many arguments


In [106]:
command_colon = re.compile(r'.*(: $: ).*')
colon_name = re.compile(r'.+(: $: )')

In [107]:
def extract_command(error):
    if command_colon.match(error):
        return re.sub(colon_name, '', error)
    else:
        return error

In [108]:
third_error["No Command Error"] = third_error["error"].apply(lambda x: extract_command(x))
third_error

,time,User Name,command,full_command,error,No Command Error
1,2021-04-19 19:35:00,User_1,export,export $TEST=Hello,PATH: $: not a valid identifier,PATH: $: not a valid identifier
2,2021-04-19 19:35:00,User_1,export,export $TEST='Hello',PATH: $: not a valid identifier,PATH: $: not a valid identifier
18,2022-09-11 18:27:00,User_3,ls,ls /home/jovyan/RNA_SEQ_SAMPLES,ls: $: No such file or directory,ls: $: No such file or directory
22,2022-09-11 18:28:00,User_3,ls,ls /gunings/RNA_SEQ_SAMPLES,ls: $: No such file or directory,ls: $: No such file or directory
29,2022-09-11 18:30:00,User_3,cd,cd sdgeorge,PATH: $: No such file or directory,PATH: $: No such file or directory
...,...,...,...,...,...,...
754,2022-12-01 20:11:00,Adam,ls,ls –l data/input/Discovery,ls: $: No such file or directory,ls: $: No such file or directory
758,2022-12-01 20:17:00,Adam,ls,ls –l ./data/input/RNA_SEQ_SAMPLES stderr.txt,ls: $: No such file or directory,ls: $: No such file or directory
759,2022-12-01 20:17:00,Adam,ls,ls –l ./data/input/RNA_SEQ_SAMPLES stderr.txt,ls: $: No such file or directory,ls: $: No such file or directory
779,2022-12-01 20:23:00,Adam,cd,cd ~/data/input/RNA_SEQ_SAMPLES/Dcd ~/data/inp...,PATH: $: too many arguments,PATH: $: too many arguments


In [109]:
permission = re.compile(r'.*(Permission).*')
io_error = re.compile(r'.*Input.output.*')
wrong_file_name = re.compile(r'.*((N|n)o such file).*')
argument_type = re.compile(r'.+a (directory|file).*')
command_name = re.compile(r'.*((C|c)ommand not found).*')
command_argu_clash = re.compile(r'.*(many|exist|same).*')

In [110]:
def replace_third(error):
    if permission.match(error):
        return "Permission Error"
    elif io_error.match(error):
        return "I/O Error"
    elif wrong_file_name.match(error):
        return "Argument Name Error"
    elif argument_type.match(error):
        return "Argument Type Error"
    elif command_name.match(error):
        return "Command Name Error"
    elif command_argu_clash.match(error):
        return "Command Argument Clash Error"
    else:
        return "Other Error"

In [111]:
third_error["Error Type"] = third_error["error"].apply(lambda x: replace_third(x))
third_error

,time,User Name,command,full_command,error,No Command Error,Error Type
1,2021-04-19 19:35:00,User_1,export,export $TEST=Hello,PATH: $: not a valid identifier,PATH: $: not a valid identifier,Other Error
2,2021-04-19 19:35:00,User_1,export,export $TEST='Hello',PATH: $: not a valid identifier,PATH: $: not a valid identifier,Other Error
18,2022-09-11 18:27:00,User_3,ls,ls /home/jovyan/RNA_SEQ_SAMPLES,ls: $: No such file or directory,ls: $: No such file or directory,Argument Name Error
22,2022-09-11 18:28:00,User_3,ls,ls /gunings/RNA_SEQ_SAMPLES,ls: $: No such file or directory,ls: $: No such file or directory,Argument Name Error
29,2022-09-11 18:30:00,User_3,cd,cd sdgeorge,PATH: $: No such file or directory,PATH: $: No such file or directory,Argument Name Error
...,...,...,...,...,...,...,...
754,2022-12-01 20:11:00,Adam,ls,ls –l data/input/Discovery,ls: $: No such file or directory,ls: $: No such file or directory,Argument Name Error
758,2022-12-01 20:17:00,Adam,ls,ls –l ./data/input/RNA_SEQ_SAMPLES stderr.txt,ls: $: No such file or directory,ls: $: No such file or directory,Argument Name Error
759,2022-12-01 20:17:00,Adam,ls,ls –l ./data/input/RNA_SEQ_SAMPLES stderr.txt,ls: $: No such file or directory,ls: $: No such file or directory,Argument Name Error
779,2022-12-01 20:23:00,Adam,cd,cd ~/data/input/RNA_SEQ_SAMPLES/Dcd ~/data/inp...,PATH: $: too many arguments,PATH: $: too many arguments,Command Argument Clash Error


In [112]:
output_dir = "error_processed_data/"
to_export_2 = third_error.copy()
to_export_2.to_pickle(output_dir + "only_error.pki")

**Third level of error counting**

In [113]:
third_error["Error Type"].value_counts()

Argument Name Error             59
Argument Type Error             11
Command Argument Clash Error     6
Other Error                      5
Command Name Error               4
Permission Error                 2
I/O Error                        1
Name: Error Type, dtype: int64

**Analyze individual category of error**

***Argument Name Error***

In [56]:
filter_argu_name = (third_error["Error Type"] == "Argument Name Error")

In [57]:
Argu_name_error = third_error[filter_argu_name]
Argu_name_error

,time,User Name,command,full_command,error,No Command Error,Error Type
18,2022-09-11 18:27:00,User_3,ls,ls /home/jovyan/RNA_SEQ_SAMPLES,ls: $: No such file or directory,ls: $: No such file or directory,Argument Name Error
22,2022-09-11 18:28:00,User_3,ls,ls /gunings/RNA_SEQ_SAMPLES,ls: $: No such file or directory,ls: $: No such file or directory,Argument Name Error
29,2022-09-11 18:30:00,User_3,cd,cd sdgeorge,PATH: $: No such file or directory,PATH: $: No such file or directory,Argument Name Error
36,2022-09-11 18:33:00,User_3,ls,"ls stdout,txt.",ls: $: No such file or directory,ls: $: No such file or directory,Argument Name Error
38,2022-09-11 18:35:00,User_3,ls,ls RNAt,ls: $: No such file or directory,ls: $: No such file or directory,Argument Name Error
39,2022-09-11 18:37:00,User_3,ls,ls RNA,ls: $: No such file or directory,ls: $: No such file or directory,Argument Name Error
40,2022-09-11 18:37:00,User_3,ls,ls RNA^t,ls: $: No such file or directory,ls: $: No such file or directory,Argument Name Error
41,2022-09-11 18:51:00,User_3,ls,ls RNA^t,ls: $: No such file or directory,ls: $: No such file or directory,Argument Name Error
51,2022-09-11 18:57:00,User_3,ls,ls –l,ls: $: No such file or directory,ls: $: No such file or directory,Argument Name Error
53,2022-09-11 19:17:00,User_3,ls,ls –l stdout.txt,ls: $: No such file or directory,ls: $: No such file or directory,Argument Name Error


In [58]:
Argu_name_error["command"].value_counts()

ls                          35
cd                           9
cat                          8
diff                         2
directory_grep_head_tail     2
print                        1
mv                           1
cp                           1
Name: command, dtype: int64

**Argument Type Error**

In [59]:
filter_argu_type = (third_error["Error Type"] == "Argument Type Error")
argu_type_error = third_error[filter_argu_type]
argu_type_error

,time,User Name,command,full_command,error,No Command Error,Error Type
186,2022-09-13 13:33:00,User_3,cat,cat cat.out goobye.txt hello_copy.txt hello_go...,cat: $: Is a directory,cat: $: Is a directory,Argument Type Error
195,2022-09-13 13:51:00,User_3,cp,cp goodbye.txt copied_ goodbye.txt,cp: target $ is not a directory,cp: target $ is not a directory,Argument Type Error
200,2022-09-13 13:56:00,User_3,ln,ln –s hello.txt linked _hello.txt,ln: target $ is not a directory,ln: target $ is not a directory,Argument Type Error
201,2022-09-13 14:04:00,User_3,cp,cp -r misc_child copied misc_child,"cp: cannot copy a directory, $","cp: cannot copy a directory, $",Argument Type Error
241,2021-04-18 22:05:00,User_4,directory_grep_head_tail,directory_grep_head_tail ~/Discovery/TSVFiles ...,PATH: $: Is a directory,PATH: $: Is a directory,Argument Type Error
241,2021-04-18 22:05:00,User_4,directory_grep_head_tail,directory_grep_head_tail ~/Discovery/TSVFiles ...,PATH: $: Is a directory,PATH: $: Is a directory,Argument Type Error
274,2021-04-19 13:23:00,User_4,gunzip,gunzip < ~/RNA_SEQ_SAMPLES/Drug_A/Drug_A_1.fas...,PATH: $: Is a directory,PATH: $: Is a directory,Argument Type Error
3,2023-01-17 19:03:00,John,/home/jovyan/data-store/data/input/RNA_SEQ_SAM...,/home/jovyan/data-store/data/input/RNA_SEQ_SAM...,PATH: $: Is a directory,PATH: $: Is a directory,Argument Type Error
115,2023-01-17 19:56:00,John,cat,cat ~/data-store/data/output/misc/*,cat: $: Is a directory,cat: $: Is a directory,Argument Type Error
138,2023-01-17 20:01:00,John,cp,cp goodbye.txt copied_ goodbye.txt,cp: target $ is not a directory,cp: target $ is not a directory,Argument Type Error


In [60]:
argu_type_error["command"].value_counts()

cp                                                    3
directory_grep_head_tail                              2
ln                                                    2
cat                                                   2
gunzip                                                1
/home/jovyan/data-store/data/input/RNA_SEQ_SAMPLES    1
Name: command, dtype: int64

**Command Name Error**

In [61]:
filter_command = (third_error["Error Type"] == "Command Name Error")

In [62]:
command_name_error = third_error[filter_command]
command_name_error

,time,User Name,command,full_command,error,No Command Error,Error Type
167,2022-09-11 20:24:00,User_3,hello.txt,hello.txt goodbye.txt,PATH: $: command not found,PATH: $: command not found,Command Name Error
169,2022-09-11 20:24:00,User_3,hello_goodbye.txt,hello_goodbye.txt,PATH: $: command not found,PATH: $: command not found,Command Name Error
390,2021-04-16 22:55:00,User_9,ll,ll misc_child/goodbye.txt,PATH: $: command not found,PATH: $: command not found,Command Name Error
395,2021-04-16 22:56:00,User_9,ll,ll,PATH: $: command not found,PATH: $: command not found,Command Name Error


In [63]:
command_name_error["command"].value_counts()

ll                   2
hello_goodbye.txt    1
hello.txt            1
Name: command, dtype: int64

**Command & Argument Clash**

In [64]:
filter_clash = (third_error["Error Type"] == "Command Argument Clash Error")
clash_error = third_error[filter_clash]
clash_error

,time,User Name,command,full_command,error,No Command Error,Error Type
177,2022-09-13 13:25:00,User_3,cd,cd misc_child/cd ..,PATH: $: too many arguments,PATH: $: too many arguments,Command Argument Clash Error
229,2021-04-18 21:01:00,User_4,ln,ln -s ~/Discovery/TSVFiles/A_2_abundance.tsv A...,ln: $: File exists,ln: $: File exists,Command Argument Clash Error
389,2021-04-16 22:55:00,User_9,cat,cat misc_child/goodbye.txt,cat: $: Too many levels of symbolic links,cat: $: Too many levels of symbolic links,Command Argument Clash Error
394,2021-04-16 22:56:00,User_9,cat,cat goodbye.txt,cat: $: Too many levels of symbolic links,cat: $: Too many levels of symbolic links,Command Argument Clash Error
428,2021-04-16 23:36:00,User_9,mv,mv link_goodbye.txt link_goodbye.txt,mv: $ are the same file,mv: $ are the same file,Command Argument Clash Error
306,2022-12-01 20:23:00,Adam,cd,cd ~/data/input/RNA_SEQ_SAMPLES/Dcd ~/data/inp...,PATH: $: too many arguments,PATH: $: too many arguments,Command Argument Clash Error


In [65]:
clash_error["command"].value_counts()

cd     2
cat    2
ln     1
mv     1
Name: command, dtype: int64